In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=e998aa91bb7176df9f82869eaed663db63e88deac69171520a25f939ebb33dad
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName('Food DF').getOrCreate()
pyspark.sql.session.SparkSession

pyspark.sql.session.SparkSession

In [ ]:
from pyspark.sql.functions import*
from pyspark.sql.types import*

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
data = spark.read.csv("drive/My Drive/Colab Notebooks/Data/indian_food.csv", header=True,inferSchema=True)
data.show()
data.printSchema()

+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|          name|         ingredients|      diet|prep_time|cook_time|flavor_profile| course|        state|region|
+--------------+--------------------+----------+---------+---------+--------------+-------+-------------+------+
|    Balu shahi|Maida flour, yogu...|vegetarian|       45|       25|         sweet|dessert|  West Bengal|  East|
|        Boondi|Gram flour, ghee,...|vegetarian|       80|       30|         sweet|dessert|    Rajasthan|  West|
|Gajar ka halwa|Carrots, milk, su...|vegetarian|       15|       60|         sweet|dessert|       Punjab| North|
|        Ghevar|Flour, ghee, kewr...|vegetarian|       15|       30|         sweet|dessert|    Rajasthan|  West|
|   Gulab jamun|Milk powder, plai...|vegetarian|       15|       40|         sweet|dessert|  West Bengal|  East|
|        Imarti|Sugar syrup, lent...|vegetarian|       10|       50|         sweet|dessert|  Wes

In [ ]:
#1. Find out how many unique dishes are present

unique_dish = data.select('name').distinct().agg(count(col('name')).alias('count')).show()

+-----+
|count|
+-----+
|  255|
+-----+



In [ ]:
#2. Which state has more dishes?

Max_dishes = data.select('state').groupBy('state').count().orderBy('count',ascending = False).show()

+---------------+-----+
|          state|count|
+---------------+-----+
|        Gujarat|   35|
|         Punjab|   32|
|    Maharashtra|   30|
|             -1|   24|
|    West Bengal|   24|
|          Assam|   21|
|     Tamil Nadu|   20|
| Andhra Pradesh|   10|
|  Uttar Pradesh|    9|
|         Kerala|    8|
|         Odisha|    7|
|      Karnataka|    6|
|      Rajasthan|    6|
|      Telangana|    5|
|            Goa|    3|
|          Bihar|    3|
| Madhya Pradesh|    2|
|        Manipur|    2|
|Jammu & Kashmir|    2|
|       Nagaland|    1|
+---------------+-----+
only showing top 20 rows



In [ ]:
#3. How many dishes from state Karnataka

karn_count = data.select('state').where(col('state')=='Karnataka').groupBy('state').count().orderBy('count', ascending = True).show()

+---------+-----+
|    state|count|
+---------+-----+
|Karnataka|    6|
+---------+-----+



In [ ]:
#4. List number of unique regions

unique_dish = data.select('region').distinct().show()

+----------+
|    region|
+----------+
|        -1|
|     South|
|   Central|
|      East|
|      West|
|North East|
|     North|
|      NULL|
+----------+



In [ ]:
#5. Count number of dishes from each region

Max_dishes = data.select('region','name').groupBy('region').count().show()

+----------+-----+
|    region|count|
+----------+-----+
|        -1|   13|
|      NULL|    1|
|     South|   59|
|   Central|    3|
|      East|   31|
|      West|   74|
|North East|   25|
|     North|   49|
+----------+-----+



In [ ]:
#6. List unique 'flavor_profile' and 'course'

unique_flavour = data.select('flavor_profile','course').distinct().show()

#unique_course = data.select('course').distinct()
#unique_course.show()

+--------------+-----------+
|flavor_profile|     course|
+--------------+-----------+
|        bitter|      snack|
|         spicy|    starter|
|          sour|main course|
|            -1|      snack|
|            -1|main course|
|         sweet|main course|
|        bitter|main course|
|         spicy|      snack|
|         sweet|    dessert|
|         spicy|main course|
+--------------+-----------+



In [ ]:
#7. Which state has more 'main course'

course = "main course"
clean_ud = data.filter(data["course"] == "main course")
state = clean_ud.groupBy("state").agg(count(col("state")).alias("count")).orderBy('count', ascending = False).show()

+---------------+-----+
|          state|count|
+---------------+-----+
|         Punjab|   28|
|     Tamil Nadu|   17|
|          Assam|   15|
|        Gujarat|   12|
|    Maharashtra|   12|
|             -1|    9|
|    West Bengal|    9|
|         Kerala|    5|
|      Karnataka|    4|
|      Rajasthan|    3|
|  Uttar Pradesh|    3|
|          Bihar|    2|
|       Nagaland|    1|
|         Odisha|    1|
| Madhya Pradesh|    1|
|        Manipur|    1|
|Jammu & Kashmir|    1|
|            Goa|    1|
|        Haryana|    1|
|   NCT of Delhi|    1|
+---------------+-----+
only showing top 20 rows



In [ ]:
course = "main course"
main = data.filter(data["course"] == course)
state = main.groupBy("state").count()
max_count = state.agg({"count": "max"}).collect()[0][0]
max_states = state.filter(state["count"] == max_count)
max_states.show()

+------+-----+
| state|count|
+------+-----+
|Punjab|   28|
+------+-----+



In [ ]:
#8. Give the %of dishes from each region

region_percent = data.groupBy("region").count().withColumn("percentage", (col("count") / data.count()) * 100)
region_percent.show()

+----------+-----+-------------------+
|    region|count|         percentage|
+----------+-----+-------------------+
|        -1|   13|  5.098039215686274|
|      NULL|    1|0.39215686274509803|
|     South|   59| 23.137254901960784|
|   Central|    3| 1.1764705882352942|
|      East|   31| 12.156862745098039|
|      West|   74|  29.01960784313726|
|North East|   25|  9.803921568627452|
|     North|   49| 19.215686274509807|
+----------+-----+-------------------+



In [ ]:
#9. List the states which has more dishes from each region

from pyspark.sql.window import Window
from pyspark.sql.functions import col,rank
d1 = data.groupBy('region','state').count()
window_spec = Window.partitionBy('region').orderBy(col('count').desc())
rank1 = d1.withColumn('Rank', rank().over(window_spec))
rank2 = rank1.filter(col('rank')==1)
rank2.drop('rank').show()


+----------+--------------+-----+
|    region|         state|count|
+----------+--------------+-----+
|      NULL| Uttar Pradesh|    1|
|        -1|            -1|   13|
|   Central|Madhya Pradesh|    2|
|      East|   West Bengal|   24|
|     North|        Punjab|   32|
|North East|         Assam|   21|
|     South|    Tamil Nadu|   20|
|      West|       Gujarat|   35|
+----------+--------------+-----+

